Multi-platform loading

In [ ]:

###SETTINGS###

import os
import subprocess
import shutil
from getpass import getpass

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "Lora")
CONTROLNET_DIR   = os.path.join(BASE_DIR, "stable-diffusion-webui", "models", "ControlNet")
EXTENSIONS_DIR   = os.path.join(BASE_DIR, "stable-diffusion-webui", "extensions")
OUTPUTS_DIR      = os.path.join(BASE_DIR, "stable-diffusion-webui", "outputs")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, EXTENSIONS_DIR, OUTPUTS_DIR]:
    os.makedirs(d, exist_ok=True)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab, then optional prompt."""
    value = os.environ.get(name)
    if value:
        return value

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            return UserSecretsClient().get_secret(name)
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value
        except Exception:
            pass

    # Manual fallback for Vast.ai/local
    try:
        return getpass(f"{name}: ").strip() or None
    except Exception:
        return None


CIVITAI_TOKEN = get_secret("CIVITAI_TOKEN")
HF_TOKEN = get_secret("HF_TOKEN")

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF"] = HF_TOKEN

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")


In [ ]:

# Download function
def download_file(url, output_path, token=None):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        return

    cmd = ["wget", "-c", "-q", url, "-O", output_path]
    if shutil.which("aria2c"):
        cmd = [
            "aria2c", "-x", "16", "-s", "16", "-k", "1M",
            "--summary-interval=0", "--console-log-level=error",
            "-d", os.path.dirname(output_path),
            "-o", os.path.basename(output_path),
            url,
        ]

    if token:
        if cmd[0] == "aria2c":
            cmd += ["--header", f"Authorization: Bearer {token}"]
        else:
            cmd = cmd[:3] + ["--header", f"Authorization: Bearer {token}"] + cmd[3:]

    try:
        subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(f"  OK → {os.path.basename(output_path)}")
    except subprocess.CalledProcessError as e:
        print(f"  ERROR → {os.path.basename(output_path)} (code {e.returncode})")
        if e.stderr and "401" in e.stderr:
            print("  NEED TOKEN")
        raise


In [ ]:
lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
]

print(f"Total LoRa in the list: {len(lora_to_download)}")
skipped = 0

print(f"All LoRA: {len(lora_to_download)}")
skipped = 0

for name, url, filename, source in lora_to_download:
    output_path = os.path.join(LORA_DIR, filename)
    
    if os.path.exists(output_path) and os.path.getsize(output_path) > 1_000_000:
        print(f"  Уже есть → {name:<30} {filename}")
        skipped += 1
        continue
    
    token = TOKENS.get(source)  # CIVITAI → CIVITAI_TOKEN value, HF → HF_TOKEN value
    
    print(f"  Loading {name:<30} ({source}) → {filename}")
    try:
        download_file(url=url, output_path=output_path, token=token)
    except Exception as e:
        print(f"  ERROR: {e}")

print(f"\nDONE. New: {len(lora_to_download) - skipped}, skipped: {skipped}")